In [ ]:
import os
import sys
import time
import logging
import multiprocessing
from typing import Dict
from typing import Dict, Generator
from typing import Dict, List, Generator

import cv2
import dask
import skimage.io
import numpy as np
import pandas as pd
import tensorflow as tf
import dask.array as da
import dask.dataframe as dd

from perceptilabs.core_new.graph import Graph
from perceptilabs.core_new.utils import Picklable
from perceptilabs.core_new.layers.base import Tf1xLayer
from perceptilabs.core_new.layers.base import DataRandom
from perceptilabs.core_new.layers.base import DataSupervised
from perceptilabs.core_new.utils import Picklable, YieldLevel
from tensorflow.python.training.tracking.base import Trackable
from perceptilabs.core_new.communication import TrainingServer
from perceptilabs.core_new.layers.base import GANLayer, Tf1xLayer
from perceptilabs.core_new.serialization import can_serialize, serialize
from perceptilabs.core_new.graph.builder import GraphBuilder, SnapshotBuilder
from perceptilabs.messaging import ZmqMessagingFactory, SimpleMessagingFactory
from perceptilabs.core_new.layers.replication import BASE_TO_REPLICA_MAP, REPLICATED_PROPERTIES_TABLE

logging.basicConfig(
    stream=sys.stdout,
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
log = logging.getLogger(__name__)




In [ ]:
# import numpy as np
class DataRandom_Random_1(DataRandom):
    """Class responsible for generating random noise"""    
    def __init__(self):
        self._variables = {}
        self._distribution = 'Normal'
        self._columns = []
        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}

    @property
    def size_training(self) -> int:
        """Returns the size of the training dataset"""                    
        return 1

    @property
    def size_validation(self) -> int:
        """Returns the size of the validation dataset"""
        return 1

    @property
    def size_testing(self) -> int:
        """Returns the size of the testing dataset"""                    
        return 1

    @property
    def variables(self) -> Dict[str, Picklable]:
        """Returns any variables that the layer should make available and that can be pickled."""
        return self._variables

    @property
    def sample(self) -> np.ndarray:
        """Returns a single data sample"""                    
        sample = next(self.make_generator_training())
        return sample

    @property
    def columns(self) -> List[str]:
        """Column names. Corresponds to each column in a sample """
        return self._columns.copy()
              
    def make_generator_training(self) -> Generator[np.ndarray, None, None]:
        """Returns a sample of random noise data."""                                        
        def gen():
            rdn_state = np.random.RandomState(1111)
            while True:
                sample = rdn_state.normal(0.1, 0.5, (100,))
                yield {'output': np.float32(sample)}
        return gen()
    
    def make_generator_testing(self) -> Generator[np.ndarray, None, None]:
        """Returns a sample of random noise data."""                                        
        def gen():
            rdn_state = np.random.RandomState(1234)
            while True:
                sample = rdn_state.normal(0.1, 0.5, (100,))
                yield {'output': np.float32(sample)}
        return gen()    
        
    def make_generator_validation(self) -> Generator[np.ndarray, None, None]:
        """Returns a sample of random noise data."""                                        
        def gen():
            rdn_state = np.random.RandomState(5678)
            while True:
                sample = rdn_state.normal(0.1, 0.5, (100,))
                yield {'output': np.float32(sample)}
        return gen()


In [ ]:
class DeepLearningFC_Dense_1(Tf1xLayer):
    def __init__(self):
        self._scope = 'DeepLearningFC_Dense_1'
        self._n_neurons = 128
        self._variables = {}
        self._keep_prob = 1

    def __call__(self, inputs: Dict[str, tf.Tensor], is_training: tf.Tensor = None) -> Dict[str, tf.Tensor]:
        """ Takes a tensor as input and feeds it forward through a layer of neurons, returning a newtensor."""
        
                            
                
        x = inputs['input']
        n_inputs = np.prod(x.get_shape().as_list()[1:], dtype=np.int32)
        b_norm = False
        y_before = None
        y = None

        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            is_training = tf.constant(True) if is_training is None else is_training
            
            initial = tf.random.truncated_normal((n_inputs, self._n_neurons), stddev=0.1)
            W = tf.compat.v1.get_variable('W', initializer=initial)

            initial = tf.constant(0., shape=[self._n_neurons])
            b = tf.compat.v1.get_variable('b', initializer=initial)
            flat_node = tf.cast(tf.reshape(x, [-1, n_inputs]), dtype=tf.float32)
            y = tf.matmul(flat_node, W) + b

            y_before = y
            

            y = tf.compat.v1.sigmoid(y)
        

            
        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}

            
        self._outputs = {            
            'output': y,
            'y_before': y_before,
            'initial': tf.expand_dims(initial, axis=0),            
            'W': tf.expand_dims(W, axis=0),            
            'b': tf.expand_dims(b, axis=0),            
            'flat_node': tf.expand_dims(flat_node, axis=0),            
        }
                            

        return self._outputs

    def get_sample(self) -> Dict[str, tf.Tensor]:
        """ Returns a dictionary of sample tensors

        Returns:
            A dictionary of sample tensors
        """
        return self._outputs

    @property
    def variables(self):
        """Any variables belonging to this layer that should be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and picklable for values.
        """
        return self._variables.copy()

    @property
    def trainable_variables(self):
        """Any trainable variables belonging to this layer that should be updated during backpropagation. Their gradients will also be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and tensors for values.
        """
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self._scope)
        variables = {v.name: v for v in variables}
        return variables

    @property
    def weights(self):
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            w = tf.compat.v1.get_variable('W')
            return {w.name: w}

    @property
    def biases(self):
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            b = tf.compat.v1.get_variable('b')
            return {b.name: b}


In [ ]:
class DeepLearningFC_Dense_2(Tf1xLayer):
    def __init__(self):
        self._scope = 'DeepLearningFC_Dense_2'
        self._n_neurons = 784
        self._variables = {}
        self._keep_prob = 1

    def __call__(self, inputs: Dict[str, tf.Tensor], is_training: tf.Tensor = None) -> Dict[str, tf.Tensor]:
        """ Takes a tensor as input and feeds it forward through a layer of neurons, returning a newtensor."""
        
                            
                
        x = inputs['input']
        n_inputs = np.prod(x.get_shape().as_list()[1:], dtype=np.int32)
        b_norm = False
        y_before = None
        y = None

        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            is_training = tf.constant(True) if is_training is None else is_training
            
            initial = tf.random.truncated_normal((n_inputs, self._n_neurons), stddev=0.1)
            W = tf.compat.v1.get_variable('W', initializer=initial)

            initial = tf.constant(0., shape=[self._n_neurons])
            b = tf.compat.v1.get_variable('b', initializer=initial)
            flat_node = tf.cast(tf.reshape(x, [-1, n_inputs]), dtype=tf.float32)
            y = tf.matmul(flat_node, W) + b

            y_before = y
            

            y = tf.compat.v1.sigmoid(y)
        

            
        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}

            
        self._outputs = {            
            'output': y,
            'y_before': y_before,
            'initial': tf.expand_dims(initial, axis=0),            
            'W': tf.expand_dims(W, axis=0),            
            'b': tf.expand_dims(b, axis=0),            
            'flat_node': tf.expand_dims(flat_node, axis=0),            
        }
                            

        return self._outputs

    def get_sample(self) -> Dict[str, tf.Tensor]:
        """ Returns a dictionary of sample tensors

        Returns:
            A dictionary of sample tensors
        """
        return self._outputs

    @property
    def variables(self):
        """Any variables belonging to this layer that should be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and picklable for values.
        """
        return self._variables.copy()

    @property
    def trainable_variables(self):
        """Any trainable variables belonging to this layer that should be updated during backpropagation. Their gradients will also be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and tensors for values.
        """
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self._scope)
        variables = {v.name: v for v in variables}
        return variables

    @property
    def weights(self):
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            w = tf.compat.v1.get_variable('W')
            return {w.name: w}

    @property
    def biases(self):
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            b = tf.compat.v1.get_variable('b')
            return {b.name: b}


In [ ]:
class ProcessReshape_Reshape_1(Tf1xLayer):
    def __call__(self, inputs: Dict[str, tf.Tensor], is_training: tf.Tensor = None) -> Dict[str, tf.Tensor]:
        """ Takes a tensor as input and reshapes it."""
        
                            
        
        shape = list((28, 28, 1))
        permutation = list((0, 1, 2))
        
        shape = [i for i in shape if i != 0]
        if(len(shape) != len(permutation)):
            permutation = []
            for i in range(len(shape)):
                permutation.append(i)

        x = inputs['input']
        input_shape = x.get_shape().as_list() 
        new_shape = [input_shape[0] if input_shape[0] is not None else -1] + shape
        
        y = tf.reshape(x, new_shape)
        y = tf.transpose(y, perm=[0] + [i+1 for i in permutation])

            
        self._outputs = {'output': y}
                

        return self._outputs

    def get_sample(self) -> Dict[str, tf.Tensor]:
        """ Returns a dictionary of sample tensors

        Returns:
            A dictionary of sample tensors
        """
        return self._outputs

    @property
    def variables(self) -> Dict[str, Picklable]:
        """Any variables belonging to this layer that should be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and picklable for values.
        """
        return {}

    @property
    def trainable_variables(self) -> Dict[str, tf.Tensor]:
        """Any trainable variables belonging to this layer that should be updated during backpropagation. Their gradients will also be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and tensors for values.
        """
        return {}
    
    @property
    def weights(self) -> Dict[str, tf.Tensor]:
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        return {}

    @property
    def biases(self) -> Dict[str, tf.Tensor]:
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        return {}


In [ ]:
class DataData_MNIST(DataSupervised):
    """Class responsible for loading data from files (e.g., numpy, csv, etc)."""    
    def __init__(self):
        self._variables = {}
        self._columns = []
        self._stored_sample = None

        columns = {}
        trn_sz_tot, val_sz_tot, tst_sz_tot = 0, 0, 0        
        trn_gens_args_DataData_MNIST, val_gens_args_DataData_MNIST, tst_gens_args_DataData_MNIST = [], [], []   
    
  

        columns_DataData_MNIST_0 = None

        global matrix_DataData_MNIST_0
        matrix_DataData_MNIST_0 = np.load("c:/users/kaeden/appdata/local/programs/python/python36/lib/site-packages/perceptilabs/tutorial_data/gan_mnist.npy", mmap_mode='r+').astype(np.float32)
        size_DataData_MNIST_0 = len(matrix_DataData_MNIST_0)

        def generator_DataData_MNIST_0(idx_lo, idx_hi):
            global matrix_DataData_MNIST_0
            yield from matrix_DataData_MNIST_0[idx_lo:idx_hi]


        if columns_DataData_MNIST_0 is not None:
            columns["DataData_MNIST_0"] = columns_DataData_MNIST_0
            self._columns = columns_DataData_MNIST_0

        trn_sz = int(round(0.01*70*size_DataData_MNIST_0))
        val_sz = int(round(0.01*20*size_DataData_MNIST_0))
        tst_sz = int(size_DataData_MNIST_0 - trn_sz - val_sz)

        if trn_sz <= 0:
            raise ValueError('Training dataset cannot be zero')
        
        if tst_sz <= 0:
            raise ValueError('Testing dataset cannot be zero')

        trn_sz_tot += trn_sz
        val_sz_tot += val_sz
        tst_sz_tot += tst_sz
        
        trn_gens_args_DataData_MNIST.append((generator_DataData_MNIST_0, 0, trn_sz))
        val_gens_args_DataData_MNIST.append((generator_DataData_MNIST_0, trn_sz, trn_sz+val_sz))
        tst_gens_args_DataData_MNIST.append((generator_DataData_MNIST_0, trn_sz+val_sz, trn_sz+val_sz+tst_sz))
                    
        self._trn_gens_args = trn_gens_args_DataData_MNIST
        self._val_gens_args = val_gens_args_DataData_MNIST                                        
        self._tst_gens_args = tst_gens_args_DataData_MNIST
                    
        self._trn_sz_tot = trn_sz_tot
        self._val_sz_tot = val_sz_tot
        self._tst_sz_tot = tst_sz_tot

        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}

    @property
    def variables(self) -> Dict[str, Picklable]:
        """Returns any variables that the layer should make available and that can be pickled."""
        return self._variables

    @property
    def sample(self) -> np.ndarray:
        """Returns a single data sample"""
        if self._stored_sample is None:                    
            self._stored_sample = next(self.make_generator_training())

        return self._stored_sample

    @property
    def columns(self) -> List[str]:
        """Column names. Corresponds to each column in a sample """
        return self._columns.copy()

    @property
    def size_training(self) -> int:
        """Returns the size of the training dataset"""                    
        return self._trn_sz_tot

    @property
    def size_validation(self) -> int:
        """Returns the size of the validation dataset"""
        return self._val_sz_tot

    @property
    def size_testing(self) -> int:
        """Returns the size of the testing dataset"""                    
        return self._tst_sz_tot
                    
    def make_generator_training(self) -> Generator[np.ndarray, None, None]:
        """Returns a generator yielding single samples of training data."""                                        
        def gen():
            for fn, lo, hi in self._trn_gens_args:
                for x in fn(lo, hi):
                    yield {'output': x}
        return gen()
        
    def make_generator_validation(self) -> Generator[np.ndarray, None, None]:
        """Returns a generator yielding single samples of validation data."""                    
        def gen():
            for fn, lo, hi in self._val_gens_args:
                for x in fn(lo, hi):
                    yield {'output': x}                    
        return gen()

    def make_generator_testing(self) -> Generator[np.ndarray, None, None]:
        """Returns a generator yielding single samples of testing data."""                            
        def gen():
            for fn, lo, hi in self._tst_gens_args:
                for x in fn(lo, hi):
                    yield {'output': x}                                        
        return gen()


In [ ]:
class ProcessReshape_Reshape_2(Tf1xLayer):
    def __call__(self, inputs: Dict[str, tf.Tensor], is_training: tf.Tensor = None) -> Dict[str, tf.Tensor]:
        """ Takes a tensor as input and reshapes it."""
        
                            
        
        shape = list((28, 28, 1))
        permutation = list((0, 1, 2))
        
        shape = [i for i in shape if i != 0]
        if(len(shape) != len(permutation)):
            permutation = []
            for i in range(len(shape)):
                permutation.append(i)

        x = inputs['input']
        input_shape = x.get_shape().as_list() 
        new_shape = [input_shape[0] if input_shape[0] is not None else -1] + shape
        
        y = tf.reshape(x, new_shape)
        y = tf.transpose(y, perm=[0] + [i+1 for i in permutation])

            
        self._outputs = {'output': y}
                

        return self._outputs

    def get_sample(self) -> Dict[str, tf.Tensor]:
        """ Returns a dictionary of sample tensors

        Returns:
            A dictionary of sample tensors
        """
        return self._outputs

    @property
    def variables(self) -> Dict[str, Picklable]:
        """Any variables belonging to this layer that should be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and picklable for values.
        """
        return {}

    @property
    def trainable_variables(self) -> Dict[str, tf.Tensor]:
        """Any trainable variables belonging to this layer that should be updated during backpropagation. Their gradients will also be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and tensors for values.
        """
        return {}
    
    @property
    def weights(self) -> Dict[str, tf.Tensor]:
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        return {}

    @property
    def biases(self) -> Dict[str, tf.Tensor]:
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        return {}


In [ ]:
class MathSwitch_Switch_1(Tf1xLayer):
    def __init__(self):
        
                            
        
        self._selected_layer_name = '1604444597887' 
        self._selected_var_name = 'input1'
        
    def __call__(self, inputs: Dict[str, tf.Tensor], is_training: tf.Tensor = None) -> Dict[str, tf.Tensor]:
        """ Takes the outputs of all the incoming layers as input and returns the output of that layer."""

        y = inputs[self._selected_var_name]
        
        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}    
        self.y = y

            
        self._outputs = {'output': y}
                

        return self._outputs

    def get_sample(self) -> Dict[str, tf.Tensor]:
        """ Returns a dictionary of sample tensors

        Returns:
            A dictionary of sample tensors
        """
        return self._outputs

    @property
    def variables(self) -> Dict[str, Picklable]:
        """Any variables belonging to this layer that should be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and picklable for values.
        """

        return self._variables.copy()

    @property
    def trainable_variables(self) -> Dict[str, tf.Tensor]:
        """Any trainable variables belonging to this layer that should be updated during backpropagation. Their gradients will also be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and tensors for values.
        """
        return {}

    @property
    def weights(self) -> Dict[str, tf.Tensor]:
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        return {}

    @property
    def biases(self) -> Dict[str, tf.Tensor]:
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        return {}


In [ ]:
class DeepLearningFC_Dense_3(Tf1xLayer):
    def __init__(self):
        self._scope = 'DeepLearningFC_Dense_3'
        self._n_neurons = 128
        self._variables = {}
        self._keep_prob = 1

    def __call__(self, inputs: Dict[str, tf.Tensor], is_training: tf.Tensor = None) -> Dict[str, tf.Tensor]:
        """ Takes a tensor as input and feeds it forward through a layer of neurons, returning a newtensor."""
        
                            
                
        x = inputs['input']
        n_inputs = np.prod(x.get_shape().as_list()[1:], dtype=np.int32)
        b_norm = False
        y_before = None
        y = None

        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            is_training = tf.constant(True) if is_training is None else is_training
            
            initial = tf.random.truncated_normal((n_inputs, self._n_neurons), stddev=0.1)
            W = tf.compat.v1.get_variable('W', initializer=initial)

            initial = tf.constant(0., shape=[self._n_neurons])
            b = tf.compat.v1.get_variable('b', initializer=initial)
            flat_node = tf.cast(tf.reshape(x, [-1, n_inputs]), dtype=tf.float32)
            y = tf.matmul(flat_node, W) + b

            y_before = y
            

            y = tf.compat.v1.sigmoid(y)
        

            
        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}

            
        self._outputs = {            
            'output': y,
            'y_before': y_before,
            'initial': tf.expand_dims(initial, axis=0),            
            'W': tf.expand_dims(W, axis=0),            
            'b': tf.expand_dims(b, axis=0),            
            'flat_node': tf.expand_dims(flat_node, axis=0),            
        }
                            

        return self._outputs

    def get_sample(self) -> Dict[str, tf.Tensor]:
        """ Returns a dictionary of sample tensors

        Returns:
            A dictionary of sample tensors
        """
        return self._outputs

    @property
    def variables(self):
        """Any variables belonging to this layer that should be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and picklable for values.
        """
        return self._variables.copy()

    @property
    def trainable_variables(self):
        """Any trainable variables belonging to this layer that should be updated during backpropagation. Their gradients will also be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and tensors for values.
        """
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self._scope)
        variables = {v.name: v for v in variables}
        return variables

    @property
    def weights(self):
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            w = tf.compat.v1.get_variable('W')
            return {w.name: w}

    @property
    def biases(self):
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            b = tf.compat.v1.get_variable('b')
            return {b.name: b}


In [ ]:
class DeepLearningFC_Is_Real(Tf1xLayer):
    def __init__(self):
        self._scope = 'DeepLearningFC_Is_Real'
        self._n_neurons = 1
        self._variables = {}
        self._keep_prob = 1

    def __call__(self, inputs: Dict[str, tf.Tensor], is_training: tf.Tensor = None) -> Dict[str, tf.Tensor]:
        """ Takes a tensor as input and feeds it forward through a layer of neurons, returning a newtensor."""
        
                            
                
        x = inputs['input']
        n_inputs = np.prod(x.get_shape().as_list()[1:], dtype=np.int32)
        b_norm = False
        y_before = None
        y = None

        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            is_training = tf.constant(True) if is_training is None else is_training
            
            initial = tf.random.truncated_normal((n_inputs, self._n_neurons), stddev=0.1)
            W = tf.compat.v1.get_variable('W', initializer=initial)

            initial = tf.constant(0., shape=[self._n_neurons])
            b = tf.compat.v1.get_variable('b', initializer=initial)
            flat_node = tf.cast(tf.reshape(x, [-1, n_inputs]), dtype=tf.float32)
            y = tf.matmul(flat_node, W) + b

            y_before = y
            

                            

            
        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}

            
        self._outputs = {            
            'output': y,
            'y_before': y_before,
            'initial': tf.expand_dims(initial, axis=0),            
            'W': tf.expand_dims(W, axis=0),            
            'b': tf.expand_dims(b, axis=0),            
            'flat_node': tf.expand_dims(flat_node, axis=0),            
        }
                            

        return self._outputs

    def get_sample(self) -> Dict[str, tf.Tensor]:
        """ Returns a dictionary of sample tensors

        Returns:
            A dictionary of sample tensors
        """
        return self._outputs

    @property
    def variables(self):
        """Any variables belonging to this layer that should be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and picklable for values.
        """
        return self._variables.copy()

    @property
    def trainable_variables(self):
        """Any trainable variables belonging to this layer that should be updated during backpropagation. Their gradients will also be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and tensors for values.
        """
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self._scope)
        variables = {v.name: v for v in variables}
        return variables

    @property
    def weights(self):
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            w = tf.compat.v1.get_variable('W')
            return {w.name: w}

    @property
    def biases(self):
        """Any weight tensors belonging to this layer that should be rendered in the frontend.

        Return:
            A dictionary with tensor names for keys and tensors for values.
        """        
        with tf.compat.v1.variable_scope(self._scope, reuse=tf.compat.v1.AUTO_REUSE):
            b = tf.compat.v1.get_variable('b')
            return {b.name: b}


In [ ]:
################################################### Main #####################################################


class TrainGan_GAN_1(GANLayer):

    def __init__(self):
        
                            
                
        self._n_epochs = 2
        self._batch_size = 100

        self._target_acc = 0
        self._stop_condition = 'Epochs'
        self._stopped = False
        self._paused = False
        self._headless = False
        self._status = 'created'
        
        self._generator_loss_training = 0.0
        self._generator_loss_validation = 0.0
        self._generator_loss_testing = 0.0   

        self._discriminator_loss_training = 0.0
        self._discriminator_loss_validation = 0.0
        self._discriminator_loss_testing = 0.0   

        self._variables = {}
        self._generator_variables = {}
        self._discriminator_variables = {}

        self._real_layer_outputs = {}
        self._generator_layer_outputs = {}
        self._generator_layer_weights = {}
        self._generator_layer_biases = {}        
        self._generator_layer_gradients = {}

        self._random_discriminator_layer_outputs = {}
        self._real_discriminator_layer_outputs = {}
        self._discriminator_layer_outputs = {}
        self._discriminator_layer_weights = {}
        self._discriminator_layer_biases = {}        
        self._discriminator_layer_gradients = {}

        self._layer_outputs = {}
        self._layer_weights = {}
        self._layer_biases = {}        
        self._layer_gradients = {}

        self._training_iteration = 0
        self._validation_iteration = 0
        self._testing_iteration = 0

        self._trn_sz_tot = 0
        self._val_sz_tot = 0
        self._tst_sz_tot = 0        
        
        self._random_means = []
        self._random_stds = []
        self._real_means = []
        self._real_stds = []

        self._checkpoint = None

    def init_layer(self, graph: Graph, mode = 'initializing'):
        """This is the function that makes the training layer runnable. We take all variable initializations for tensors and initializers and wrap them in dictionaries
        to be called in run().
        """
        self._mode = mode
        if tf.compat.v1.get_default_session() is not None:
            tf.compat.v1.get_default_session().close() 
            tf.reset_default_graph()
        self._status = 'initializing'

        real_data_layer_id = "DataData_MNIST"

        switch_layer_id = "MathSwitch_Switch_1"

        for node in graph.data_nodes:
            if node.layer_id != real_data_layer_id and not node.is_training_node:
                random_data_node = node
            elif node.layer_id == real_data_layer_id:
                real_data_node = node

        training_node = graph.nodes[-1]
        switch_node = graph.get_node_by_id(switch_layer_id)
        
        output_node = [node for node in graph.get_input_nodes(training_node)][0]
        output_layer_id = output_node.layer_id
        self._switch_layer_id = switch_layer_id

        self._selected_layer_id = "1604444597887"
        
        generator_nodes = graph.get_nodes_inbetween(random_data_node, switch_node)
        discriminator_nodes = graph.get_nodes_inbetween(switch_node, output_node)
        real_nodes = graph.get_nodes_inbetween(real_data_node, switch_node)

        generator_layer_ids = [node.layer_id for node in generator_nodes]
        discriminator_layer_ids = [node.layer_id for node in discriminator_nodes]

        self._generator_layer_ids = generator_layer_ids

        self._trn_sz_tot = real_data_node.layer.size_training
        self._val_sz_tot = real_data_node.layer.size_validation
        self._tst_sz_tot = real_data_node.layer.size_testing

        real_data_sample = real_data_node.layer_instance.sample
        random_data_sample = random_data_node.layer_instance.sample 
        # Make training set
        dataset_trn = tf.data.Dataset.zip((
            tf.data.Dataset.from_generator(
                real_data_node.layer_instance.make_generator_training,
                output_shapes={k: v.shape for k, v in real_data_sample.items()},
                output_types={k: v.dtype for k, v in real_data_sample.items()}                
            ),
            tf.data.Dataset.from_generator(
                random_data_node.layer_instance.make_generator_training,
                output_shapes={k: v.shape for k, v in random_data_sample.items()},
                output_types={k: v.dtype for k, v in random_data_sample.items()}
            )
        ))

        # Make validation set
        dataset_val = tf.data.Dataset.zip((
            tf.data.Dataset.from_generator(
                real_data_node.layer_instance.make_generator_validation,
                output_shapes={k: v.shape for k, v in real_data_sample.items()},
                output_types={k: v.dtype for k, v in real_data_sample.items()}                
            ),
            tf.data.Dataset.from_generator(
                random_data_node.layer_instance.make_generator_validation,
                output_shapes={k: v.shape for k, v in random_data_sample.items()},
                output_types={k: v.dtype for k, v in random_data_sample.items()}
            )
        ))

        # Make testing set
        dataset_tst = tf.data.Dataset.zip((
            tf.data.Dataset.from_generator(
                real_data_node.layer_instance.make_generator_testing,
                output_shapes={k: v.shape for k, v in real_data_sample.items()},
                output_types={k: v.dtype for k, v in real_data_sample.items()}             
            ),
            tf.data.Dataset.from_generator(
                random_data_node.layer_instance.make_generator_testing,
                output_shapes={k: v.shape for k, v in random_data_sample.items()},
                output_types={k: v.dtype for k, v in random_data_sample.items()}
            )
        ))

        dataset_trn = dataset_trn.batch(self._batch_size)
        dataset_val = dataset_val.batch(self._batch_size)
        dataset_tst = dataset_tst.batch(1)

        # Make initializers
        iterator = tf.data.Iterator.from_structure(dataset_trn.output_types, dataset_trn.output_shapes)
        trn_init = iterator.make_initializer(dataset_trn)
        val_init = iterator.make_initializer(dataset_val)
        tst_init = iterator.make_initializer(dataset_tst)        
        real_tensor , random_tensor = iterator.get_next()

        

        # Build the TensorFlow graph
        def build_real_graph(real_tensor):
            if len(real_nodes) > 1:
                switch_node.layer._selected_layer_id = real_nodes[-2].layer_id
                switch_node.layer._selected_var_name = [dst_var for src_node, src_var, dst_var in graph.get_input_connections(switch_node) if src_node.layer_id == switch_node.layer._selected_layer_id][0]
            else:
                switch_node.layer._selected_layer_id = real_data_node.layer_id
                switch_node.layer._selected_var_name = [dst_var for src_node, src_var, dst_var in graph.get_input_connections(switch_node) if src_node.layer_id == switch_node.layer._selected_layer_id][0]

            _random_data_node=random_data_node

            layer_output_tensors = {
                real_data_node.layer_id: real_tensor
            }

            for dst_node in real_nodes:
                inputs = {
                    dst_var: layer_output_tensors[src_node.layer_id][src_var]
                    for src_node, src_var, dst_var in graph.get_input_connections(dst_node)
                    if src_node not in generator_nodes + [_random_data_node]
                }
                y = dst_node.layer_instance(
                    inputs
                )
                layer_output_tensors[dst_node.layer_id] = y

            return layer_output_tensors

        def build_generator_graph(random_tensor):
            switch_node.layer._selected_layer_id = generator_nodes[-2].layer_id
            switch_node.layer._selected_var_name = [dst_var for src_node, src_var, dst_var in graph.get_input_connections(switch_node) if src_node.layer_id == switch_node.layer._selected_layer_id][0]
            _real_data_node = real_data_node

            layer_output_tensors = {
                random_data_node.layer_id: random_tensor
            }

            for dst_node in generator_nodes:
                inputs = {
                    dst_var: layer_output_tensors[src_node.layer_id][src_var]
                    for src_node, src_var, dst_var in graph.get_input_connections(dst_node) 
                    if src_node not in real_nodes+[_real_data_node]
                }
                y = dst_node.layer_instance(
                    inputs
                )
                layer_output_tensors[dst_node.layer_id] = y



            # for node in generator_nodes:
            #     if len(list(graph.get_input_nodes(node))) <= 1:
            #         args = []
            #         for input_node in graph.get_input_nodes(node):
            #             args.append(layer_output_tensors[input_node.layer_id])
            #         y = node.layer_instance(*args)
            #     elif len(list(graph.get_input_nodes(node))) > 1:
            #         args = {}
            #         for input_node in graph.get_input_nodes(node):
            #             if input_node in generator_nodes:
            #                 args[input_node.layer_id] = layer_output_tensors[input_node.layer_id]
            #         y = node.layer_instance(args)
            #     layer_output_tensors[node.layer_id] = y
            return layer_output_tensors
        
        def build_discriminator_graph(input_tensor):
            layer_output_tensors = {
                switch_node.layer_id: input_tensor
            }
            for dst_node in discriminator_nodes:
                inputs = {
                    dst_var: layer_output_tensors[src_node.layer_id][src_var]
                    for src_node, src_var, dst_var in graph.get_input_connections(dst_node)
                }
                y = dst_node.layer_instance(
                    inputs
                )
                layer_output_tensors[dst_node.layer_id] = y


            return layer_output_tensors

        generator_layer_output_tensors = build_generator_graph(random_tensor)
        real_layer_output_tensors = build_real_graph(real_tensor)
        
        real_output_tensor = real_layer_output_tensors[switch_layer_id]
        generator_output_tensor = generator_layer_output_tensors[switch_layer_id]
        
        real_discriminator_layer_output_tensors = build_discriminator_graph(real_output_tensor)
        random_discriminator_layer_output_tensors = build_discriminator_graph(generator_output_tensor)
        
        # real_discriminator_outputs = real_discriminator_layer_output_tensors[output_layer_id]
        # random_discriminator_outputs = random_discriminator_layer_output_tensors[output_layer_id]
        
        # Create an exportable version of the TensorFlow graph
        
        # self._real_tensor_export = tf.placeholder(shape=dataset_trn.output_shapes[0], dtype=dataset_trn.output_types[0])
        # self._random_tensor_export = tf.placeholder(shape=dataset_trn.output_shapes[1], dtype=dataset_trn.output_types[1])

        self._real_tensor_export= {
            key: tf.placeholder(shape=shape, dtype=type_)
            for (key, shape), (_, type_) in zip(dataset_trn.output_shapes[0].items(), dataset_trn.output_types[0].items())            
        }
        self._random_tensor_export= {
            key: tf.placeholder(shape=shape, dtype=type_)
            for (key, shape), (_, type_) in zip(dataset_trn.output_shapes[1].items(), dataset_trn.output_types[1].items())            
        }
        

        self._real_output_tensor_export = build_real_graph(self._real_tensor_export)[switch_layer_id]
        self._generator_output_tensor_export = build_generator_graph(self._random_tensor_export)[switch_layer_id]

        self._discriminator_real_output_tensor_export = build_discriminator_graph(self._real_output_tensor_export)[output_layer_id]
        self._discriminator_random_output_tensor_export = build_discriminator_graph(self._generator_output_tensor_export)[output_layer_id]
        
        #loss function

        def loss_func(logits_in,labels_in):
            return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels_in))

        real_discriminator_output_tensor = None
        random_discriminator_output_tensor = None
        
        for src_node, src_var, dst_var in graph.get_input_connections(graph.active_training_node):
            if dst_var == 'input':
                real_discriminator_output_tensor = real_discriminator_layer_output_tensors[output_layer_id][src_var]
                random_discriminator_output_tensor = random_discriminator_layer_output_tensors[output_layer_id][src_var]
        
        discriminator_real_loss=loss_func(real_discriminator_output_tensor,tf.ones_like(real_discriminator_output_tensor)*0.9) 
        discriminator_random_loss=loss_func(random_discriminator_output_tensor,tf.zeros_like(random_discriminator_output_tensor))
        
        discriminator_loss_tensor = discriminator_real_loss+discriminator_random_loss
        generator_loss_tensor = loss_func(random_discriminator_output_tensor,tf.ones_like(random_discriminator_output_tensor))

        global_step = None

        generator_optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999)


        discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999)

        generator_layer_weight_tensors = {}
        generator_layer_bias_tensors = {}        
        generator_layer_gradient_tensors = {}

        for node in generator_nodes+real_nodes:
            if not isinstance(node.layer, Tf1xLayer): # In case of pure custom layers...
                continue
            
            generator_layer_weight_tensors[node.layer_id] = node.layer.weights
            generator_layer_bias_tensors[node.layer_id] = node.layer.biases
            
            if len(node.layer.trainable_variables) > 0:
                gradients = {}
                for name, tensor in node.layer.trainable_variables.items():
                    grad_tensor = tf.gradients(generator_loss_tensor, tensor)
                    if any(x is None for x in grad_tensor):
                        grad_tensor = tf.constant(0)
                    gradients[name] = grad_tensor
                generator_layer_gradient_tensors[node.layer_id] = gradients

        discriminator_layer_weight_tensors = {}
        discriminator_layer_bias_tensors = {}        
        discriminator_layer_gradient_tensors = {}

        for node in discriminator_nodes:
            if not isinstance(node.layer, Tf1xLayer): # In case of pure custom layers...
                continue
            
            discriminator_layer_weight_tensors[node.layer_id] = node.layer.weights
            discriminator_layer_bias_tensors[node.layer_id] = node.layer.biases
            
            if len(node.layer.trainable_variables) > 0:
                gradients = {}
                for name, tensor in node.layer.trainable_variables.items():
                    grad_tensor = tf.gradients(discriminator_loss_tensor, tensor)
                    if any(x is None for x in grad_tensor):
                        grad_tensor = tf.constant(0)
                    gradients[name] = grad_tensor
                discriminator_layer_gradient_tensors[node.layer_id] = gradients

        
        trainable_vars = tf.trainable_variables()
        
        discriminator_vars=[]
        for var in trainable_vars:
            for discriminator_layer_id in discriminator_layer_ids:
                if discriminator_layer_id in var.name:
                    discriminator_vars.append(var)

        discriminator_update_weights = discriminator_optimizer.minimize(discriminator_loss_tensor, var_list = discriminator_vars, global_step=global_step)

        generator_vars=[]
        for var in trainable_vars:
            for generator_layer_id in generator_layer_ids:
                if generator_layer_id in var.name:
                    generator_vars.append(var)

        generator_update_weights = generator_optimizer.minimize(generator_loss_tensor, var_list = generator_vars, global_step=global_step) 
             

        sess = None
         
        config = tf.ConfigProto(device_count={"GPU": 0})
        sess = tf.Session(config=config)
      
        self._sess = sess
        
        trackable_variables = {}
        trackable_variables.update({x.name: x for x in tf.trainable_variables() if isinstance(x, Trackable)})
        trackable_variables.update({k: v for k, v in locals().items() if isinstance(v, Trackable) and not isinstance(v, tf.python.data.ops.iterator_ops.Iterator)})
        self._checkpoint = tf.train.Checkpoint(**trackable_variables)
        sess.run(tf.global_variables_initializer())
        
        checkpoint_directory = '//192.168.1.31/UserFiles/Kaeden/My Documents/Perceptilabs/Default/GAN_Test/checkpoint'
        use_checkpoint = False
        if use_checkpoint:
            path = tf.train.latest_checkpoint(checkpoint_directory)
            if path is not None:
                status = self._checkpoint.restore(path)
                status.run_restore_ops(session=self._sess)
            elif path is None and self._mode == 'testing':
                log.error('There are no saved checkpoint files for this model.')
                self._sess.close()


        self._discriminator_layer_gradient = discriminator_layer_gradient_tensors
        self._discriminator_layer_bias = discriminator_layer_bias_tensors
        self._discriminator_layer_weight = discriminator_layer_weight_tensors.copy()
        self._real_discriminator_layer_output = real_discriminator_layer_output_tensors
        self._random_discriminator_layer_output = random_discriminator_layer_output_tensors
        self._discriminator_loss = discriminator_loss_tensor
        self._discriminator_weight = discriminator_update_weights
        self._real_layer_output = real_layer_output_tensors
        self._gen_layer_gradient = generator_layer_gradient_tensors
        self._gen_layer_bias = generator_layer_bias_tensors
        self._gen_layer_weight = generator_layer_weight_tensors.copy()
        self._gen_layer_output = generator_layer_output_tensors.copy()
        self._gen_loss = generator_loss_tensor
        self._gen_weight = generator_update_weights

        self._trn_init = trn_init
        self._val_init = val_init
        self._tst_init = tst_init

        
    def train(self, graph:Graph):
        """Training is done when this function is called. Once the training ends, checkpoint files are saved.
        """
    
        sess = self._sess
        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}
        self._checkpoint_save_path = '//192.168.1.31/UserFiles/Kaeden/My Documents/Perceptilabs/Default/GAN_Test/checkpoint'


        def train_step(sess):
            if not self._headless:
                _, self._generator_loss_training, self._generator_layer_outputs,\
                    self._generator_layer_weights, self._generator_layer_biases, \
                    self._generator_layer_gradients, self._real_layer_outputs, \
                _, self._discriminator_loss_training, self._random_discriminator_layer_outputs, self._real_discriminator_layer_outputs,\
                    self._discriminator_layer_weights, self._discriminator_layer_biases, \
                    self._discriminator_layer_gradients \
                    = sess.run([
                        self._gen_weight, self._gen_loss, self._gen_layer_output, \
                        self._gen_layer_weight, self._gen_layer_bias, self._gen_layer_gradient, self._real_layer_output, \
                        self._discriminator_weight, self._discriminator_loss, self._random_discriminator_layer_output, \
                        self._real_discriminator_layer_output, self._discriminator_layer_weight, self._discriminator_layer_bias, self._discriminator_layer_gradient
                    ])
                
            else:
                _, self._generator_loss_training, \
                    _, self.__discriminator_loss_training \
                    = sess.run([
                        self._gen_weight, self._gen_loss, \
                            self._discriminator_weight, self._discriminator_loss
                    ])

        def validation_step(sess):
            if not self._headless:
                self._generator_loss_validation, self._generator_layer_outputs,\
                    self._generator_layer_weights, self._generator_layer_biases, \
                    self._generator_layer_gradients, self._real_layer_outputs, \
                    self._discriminator_loss_validation, self._discriminator_layer_outputs,\
                    self._discriminator_layer_weights, self._discriminator_layer_biases, \
                    self._discriminator_layer_gradients \
                    = sess.run([
                        self._gen_loss, self._gen_layer_output, \
                        self._gen_layer_weight, self._gen_layer_bias, self._gen_layer_gradient, self._real_layer_output, \
                        self._discriminator_loss, self._random_discriminator_layer_output, \
                        self._discriminator_layer_weight, self._discriminator_layer_bias, self._discriminator_layer_gradient
                    ])

            else:
                self._generator_loss_validation, \
                    self.__discriminator_loss_validation \
                    = sess.run(
                        self._gen_loss, \
                        self._discriminator_loss
                    )

        log.info("Entering training loop")

        # Training loop
        self._epoch = 0
        while self._epoch < self._n_epochs and not self._stopped:
            t0 = time.perf_counter()
            self._training_iteration = 0
            self._validation_iteration = 0
            self._status = 'training'
            sess.run(self._trn_init)            
            try:
                while not self._stopped:
                    train_step(sess)
                    yield YieldLevel.SNAPSHOT
                    self._training_iteration += 1
            except tf.errors.OutOfRangeError:
                pass

            self._status = 'validation'
            sess.run(self._val_init)            
            try:
                while not self._stopped:
                    validation_step(sess)
                    yield YieldLevel.SNAPSHOT                    
                    self._validation_iteration += 1
            except tf.errors.OutOfRangeError:
                pass
            log.info(
                f"Finished epoch {self._epoch+1}/{self._n_epochs} - "
                f"generator loss training, validation: {self.generator_loss_training:.6f}, {self.generator_loss_validation:.6f} - "
                f"discriminator loss training, validation: {self.discriminator_loss_training:.6f}, {self.discriminator_loss_validation:.6f} - "
            )
            log.info(f"Epoch duration: {round(time.perf_counter() - t0, 3)} s")  
            if self._stop_condition == "TargetAccuracy" and self._accuracy_training * 100 >= self._target_acc:
                break          
            self._epoch += 1

        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}            
        self._status = 'finished'
        yield YieldLevel.SNAPSHOT
        self.on_export(self._checkpoint_save_path, 'checkpoint')   
        sess.close()

 

    def test(self, graph:Graph):
        """Testing is done when this function is called. 
        """

    
        
        sess = self._sess
        self._epoch = 0

        def test_step(sess):
            self._generator_loss_testing, self._generator_layer_outputs,\
                self._generator_layer_weights, self._generator_layer_gradients, self._real_layer_outputs, \
                self._discriminator_loss_testing, self._discriminator_layer_outputs,\
                self._discriminator_layer_weights, self._discriminator_layer_gradients \
                = sess.run([
                    self._gen_loss, self._gen_layer_output, \
                    self._gen_layer_weight, self._gen_layer_gradient, self._real_layer_output, \
                    self._discriminator_loss, self._random_discriminator_layer_output, \
                    self._discriminator_layer_weight, self._discriminator_layer_gradient
                ])

        # Test loop
        log.info("Entering testing loop")
        self._status = 'testing'

        self._testing_iteration = 0
        sess.run(self._tst_init)                                
        while not self._stopped:
            try:
                test_step(sess)
                yield YieldLevel.SNAPSHOT
                self._testing_iteration += 1
            except tf.errors.OutOfRangeError:
                self._testing_iteration = 0
                sess.run(self._tst_init)
                test_step(sess)   
                yield YieldLevel.SNAPSHOT
        
        self._status = 'finished'
        self._variables = {k: v for k, v in locals().items() if can_serialize(v)}
        yield YieldLevel.SNAPSHOT
        sess.close()

 

    def run(self, graph: Graph, mode = 'initializing'):
        """Called as the main entry point for training. Responsible for training the model.

        Args:
            graph: A PerceptiLabs Graph object containing references to all layers objects included in the model produced by this training layer.
        """  
        self.init_layer(graph, mode)
        self._variables = {k: v for k, v in locals().items() if can_serialize(v)} 
        
        if mode == 'training':
            yield from self.train(graph)
        elif mode == 'testing':
            yield from self.test(graph)


    def on_export(self, path: str, mode: str) -> None:
        """Called when the export button is clicked in the frontend.
        It is up to the implementing layer to save the model to disk.
        
        Args:
            path: the directory where the exported model will be stored.
            mode: how to export the model. Made available to frontend via 'export_modes' property."""

        log.debug(f"Export called. Project path = {path}, mode = {mode}")
        
        if mode in ['TFModel', 'TFLite']:
            pb_path = os.path.join(path, '1')
            if os.path.exists(pb_path):
                shutil.rmtree(pb_path)
            
            time.sleep(.0000000000000001) #Force your computer to do a clock cycle to avoid Windows permission exception

            os.makedirs(pb_path, exist_ok=True)
        
        # Export non-compressed model
        if mode in ['TFModel']:
            tf.compat.v1.saved_model.simple_save(self._sess, pb_path, \
                inputs={'real_input': self._real_tensor_export['output'], 'random_input': self._random_tensor_export['output']}, \
                outputs={'real_output': self._discriminator_real_output_tensor_export['output'], 'random_output': self._discriminator_random_output_tensor_export['output'] })

        # Export compressed model
        if mode in ['TFLite']:
            frozen_path = os.path.join(pb_path, 'frozen_model.pb')
            converter = tf.lite.TFLiteConverter.from_session(self._sess, [self._real_tensor_export['output'], self._random_tensor_export['output']], [self._discriminator_real_output_tensor_export['output'], self._discriminator_random_output_tensor_export['output']])
            converter.post_training_quantize = True
            tflite_model = converter.convert()
            with open(frozen_path, "wb") as f:
                f.write(tflite_model)

        # Export checkpoint
        if mode in ['checkpoint']:
            for fname in os.listdir(path):
                if fname.endswith('.json'):
                    pass
                else:
                    os.remove(os.path.join(path,fname))
            self._checkpoint.save(file_prefix=os.path.join(path, 'model.ckpt'), session=self._sess)
                
    def on_stop(self) -> None:
        """Called when the save model button is clicked in the frontend. 
        It is up to the implementing layer to save the model to disk."""
        self.on_export(self._checkpoint_save_path, 'checkpoint')   
        self._stopped = True

    def on_headless_activate(self) -> None:
        """"Called when the statistics shown in statistics window are not needed.
        Purose is to speed up the iteration speed significantly."""
        self._headless = True

        self._layer_outputs = {} 
        self._layer_weights = {}
        self._layer_biases = {}
        self._layer_gradients = {}

    def on_headless_deactivate(self) -> None:
        """"Called when the statistics shown in statistics window are needed.
        May slow down the iteration speed of the training."""
        import time
        log.info(f"Set to headless_off at time {time.time()}")
        self._headless = False

    @property
    def export_modes(self) -> List[str]:
        """Returns the possible modes of exporting a model."""        
        return [
            'TFModel',
            'TFLite'
            'TFModel+checkpoint',
            'TFLite+checkpoint',            
        ]
        
    @property
    def is_paused(self) -> None:
        """Returns true when the training is paused."""        
        return self._paused

    @property
    def batch_size(self):
        """ Size of the current training batch """        
        return self._batch_size

    @property
    def status(self):
        """Called when the pause button is clicked in the frontend. It is up to the implementing layer to pause its execution."""        
        return self._status
    
    @property
    def epoch(self):
        """The current epoch"""        
        return self._epoch

    @property
    def variables(self):
        """Any variables belonging to this layer that should be rendered in the frontend.
        
        Returns:
            A dictionary with tensor names for keys and picklable for values.
        """
        return self._variables.copy()        

    @property
    def sample(self) -> Dict[str, Dict[str, Picklable]]:
        """Returns a single data sample"""       
        sample = np.array([self._generator_loss_training])
        return {'output': sample}

    @property
    def columns(self) -> List[str]: 
        """Column names. Corresponds to each column in a sample """
        return []
    
    @property
    def size_training(self) -> int:
        """Returns the size of the training dataset"""                                    
        return self._trn_sz_tot

    @property
    def size_validation(self) -> int:
        """Returns the size of the validation dataset"""                                            
        return self._val_sz_tot

    @property
    def size_testing(self) -> int:
        """Returns the size of the testing dataset"""
        return self._tst_sz_tot

    def make_generator_training(self) -> Generator[np.ndarray, None, None]:
        """Returns a generator yielding single samples of training data. In the case of a training layer, this typically yields the model output."""        
        yield from []
        
    def make_generator_validation(self) -> Generator[np.ndarray, None, None]:
        """Returns a generator yielding single samples of validation data. In the case of a training layer, this typically yields the model output."""                
        yield from []
        
    def make_generator_testing(self) -> Generator[np.ndarray, None, None]:
        """Returns a generator yielding single samples of testing data. In the case of a training layer, this typically yields the model output."""                        
        yield from []


    @property
    def get_switch_layer_id(self):
        """ Returns the layer name of the Switch layer"""
        return self._switch_layer_id
    
    @property
    def generator_loss_training(self) -> float:
        """Returns the current loss of the generator training phase"""                
        return self._generator_loss_training        

    @property
    def generator_loss_validation(self) -> float:
        """Returns the current loss of the generator validation phase"""                        
        return self._generator_loss_validation        

    @property
    def generator_loss_testing(self) -> float:
        """Returns the current loss of the generator testing phase"""                
        return self._generator_loss_testing
    
    @property
    def discriminator_loss_validation(self) -> float:
        """Returns the current loss of the discriminator validation phase"""                        
        return self._discriminator_loss_validation        

    @property
    def discriminator_loss_training(self) -> float:
        """Returns the current loss of the discriminator testing phase"""                
        return self._discriminator_loss_training

    @property
    def discriminator_loss_testing(self) -> float:
        """Returns the current loss of the discriminator testing phase"""                
        return self._discriminator_loss_testing

    @property
    def layer_weights(self) -> Dict[str, Dict[str, Picklable]]:
        """The weight values of each layer in the input Graph during the training.

        Returns:
            A dictionary of nested dictionaries, where each key is a layer id. The nested dictionaries contain weight name and value pairs. The values must be picklable.
        """      
        self._layer_weights.update(self._generator_layer_weights)
        self._layer_weights.update(self._discriminator_layer_weights)   
        return self._layer_weights

    @property
    def layer_biases(self) -> Dict[str, Dict[str, Picklable]]:
        """The bias values of each layer in the input Graph during the training.

        Returns:
            A dictionary of nested dictionaries, where each key is a layer id. The nested dictionaries contain weight name and value pairs. The values must be picklable.
        """       
        self._layer_biases.update(self._generator_layer_biases)
        self._layer_biases.update(self._discriminator_layer_biases) 
        return self._layer_biases
    
    @property
    def layer_gradients(self) -> Dict[str, Dict[str, Picklable]]:
        """The gradients with respect to the loss of all trainable variables of each layer in the input Graph.

        Returns:
            A dictionary of nested dictionaries, where each key is a layer id. The nested dictionaries contain gradient name and value pairs. The values must be picklable.
        """
        self._layer_gradients.update(self._generator_layer_gradients)
        self._layer_gradients.update(self._discriminator_layer_gradients)       
        return self._layer_gradients
    
    @property
    def layer_outputs(self) -> Dict[str, Dict[str, Picklable]]:
        """The output values of each layer in the input Graph during the training (e.g., tf.Tensors evaluated for each iteration)

        Returns:
            A dictionary of nested dictionaries, where each key is a layer id. The nested dictionaries contain variable name and value pairs. The values must be picklable.
        """
        if self._selected_layer_id in self._generator_layer_ids:
            self._layer_outputs.update(self._real_layer_outputs)
            self._layer_outputs.update(self._generator_layer_outputs)
            self._layer_outputs.update(self._random_discriminator_layer_outputs)
        else:
            self._layer_outputs.update(self._generator_layer_outputs)
            self._layer_outputs.update(self._real_layer_outputs)
            self._layer_outputs.update(self._real_discriminator_layer_outputs)
        return self._layer_outputs

    @property
    def generator_layer_outputs(self) -> Dict[str, Dict[str, Picklable]]:
        """The output values of each layer in the input Graph during the training (e.g., tf.Tensors evaluated for each iteration)

        Returns:
            A dictionary of nested dictionaries, where each key is a layer id. The nested dictionaries contain variable name and value pairs. The values must be picklable.
        """
        return self._generator_layer_outputs
    
    @property
    def real_layer_outputs(self) -> Dict[str, Dict[str, Picklable]]:
        """The output values of each layer in the input Graph during the training (e.g., tf.Tensors evaluated for each iteration)

        Returns:
            A dictionary of nested dictionaries, where each key is a layer id. The nested dictionaries contain variable name and value pairs. The values must be picklable.
        """
        return self._real_layer_outputs

    @property
    def training_iteration(self) -> int:
        """The current training iteration"""
        return self._training_iteration

    @property
    def validation_iteration(self) -> int:
        """The current validation iteration"""        
        return self._validation_iteration

    @property
    def testing_iteration(self) -> int:
        """The current testing iteration"""                
        return self._testing_iteration
    
    @property
    def progress(self) -> float:
        """A number indicating the overall progress of the training
        
        Returns:
            A floating point number between 0 and 1
        """        
        n_iterations_per_epoch = np.ceil(self.size_training / self.batch_size) + \
                                 np.ceil(self.size_validation / self.batch_size)
        n_iterations_total = self._n_epochs * n_iterations_per_epoch

        iteration = self.epoch * n_iterations_per_epoch + \
                    self.training_iteration + self.validation_iteration
        
        progress = min(iteration/(n_iterations_total - 1), 1.0)
        return progress


In [ ]:
layer_classes = {
    'DataRandom_Random_1': DataRandom_Random_1,
    'DeepLearningFC_Dense_1': DeepLearningFC_Dense_1,
    'DeepLearningFC_Dense_2': DeepLearningFC_Dense_2,
    'ProcessReshape_Reshape_1': ProcessReshape_Reshape_1,
    'DataData_MNIST': DataData_MNIST,
    'ProcessReshape_Reshape_2': ProcessReshape_Reshape_2,
    'MathSwitch_Switch_1': MathSwitch_Switch_1,
    'DeepLearningFC_Dense_3': DeepLearningFC_Dense_3,
    'DeepLearningFC_Is_Real': DeepLearningFC_Is_Real,
    'TrainGan_GAN_1': TrainGan_GAN_1,
}

edges = {
    ('DataRandom_Random_1', 'DeepLearningFC_Dense_1'),
    ('DeepLearningFC_Dense_1', 'DeepLearningFC_Dense_2'),
    ('DeepLearningFC_Dense_2', 'ProcessReshape_Reshape_1'),
    ('ProcessReshape_Reshape_1', 'MathSwitch_Switch_1'),
    ('DataData_MNIST', 'ProcessReshape_Reshape_2'),
    ('ProcessReshape_Reshape_2', 'MathSwitch_Switch_1'),
    ('MathSwitch_Switch_1', 'DeepLearningFC_Dense_3'),
    ('DeepLearningFC_Dense_3', 'DeepLearningFC_Is_Real'),
    ('DeepLearningFC_Is_Real', 'TrainGan_GAN_1'),
}

conn_info = {
    'DataRandom_Random_1:DeepLearningFC_Dense_1': [('output', 'input')],
    'DeepLearningFC_Dense_1:DeepLearningFC_Dense_2': [('output', 'input')],
    'DeepLearningFC_Dense_2:ProcessReshape_Reshape_1': [('output', 'input')],
    'ProcessReshape_Reshape_1:MathSwitch_Switch_1': [('output', 'input1')],
    'DataData_MNIST:ProcessReshape_Reshape_2': [('output', 'input')],
    'ProcessReshape_Reshape_2:MathSwitch_Switch_1': [('output', 'input2')],
    'MathSwitch_Switch_1:DeepLearningFC_Dense_3': [('output', 'input')],
    'DeepLearningFC_Dense_3:DeepLearningFC_Is_Real': [('output', 'input')],
    'DeepLearningFC_Is_Real:TrainGan_GAN_1': [('output', 'input')],
}


iterator = graph.training_nodes[0].layer_instance.run(graph)
result = None
sentinel = object()
while result is not sentinel:
    result = next(iterator, sentinel)

